# 시행착오

In [6]:
import requests, bs4
import pandas as pd
from lxml import html #html.lxml 로 써진 것. 즉 html.lxml 로 써야하는 명령어를 lxml만써도 작동하게해줌.
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from pprint import pprint
import numpy as np

In [4]:
%ls trimdata

 Volume in drive C has no label.
 Volume Serial Number is 04ED-7805

 Directory of c:\projects\python\apt\trimdata

2021-04-15  오전 01:00    <DIR>          .
2021-04-15  오전 01:00    <DIR>          ..
2021-04-15  오전 12:29         1,600,714 apt200601.xlsx
2021-04-15  오전 12:30         2,634,168 apt200602.xlsx
2021-04-15  오전 12:30         3,389,704 apt200603.xlsx
2021-04-15  오전 12:30         2,942,021 apt200604.xlsx
2021-04-15  오전 12:31         2,780,836 apt200605.xlsx
2021-04-15  오전 12:31         2,219,198 apt200606.xlsx
2021-04-15  오전 12:31         2,036,597 apt200607.xlsx
2021-04-15  오전 12:31         2,531,730 apt200608.xlsx
2021-04-15  오전 12:32         3,993,229 apt200609.xlsx
2021-04-15  오전 12:48         4,785,614 apt200610.xlsx
2021-04-15  오전 12:49         4,761,955 apt200611.xlsx
2021-04-15  오전 12:49         3,362,008 apt200612.xlsx
2021-04-15  오전 12:32         2,388,204 apt200701.xlsx
2021-04-15  오전 12:33         2,159,101 apt200702.xlsx
2021-04-15  오전 12:33         2,711,712 apt200

In [5]:
# df_09 = pd.read_csv("APTdata/아파트(매매)__실거래가_202010.csv",encoding="cp949")
# #에러났음. 왜그렇지? pd import를안함. 
# Parse error. . 이전 강의에서 배웠죠? 근데 sep을 바꿔줬었는데 이번엔 위에 내용이 있어서 안될수도있음.

# 일단 좆된것. csv는 파일지워서 다른이름으로저장하면 손상이되어버림.
# df = pd.read_csv("APTdata/아파트(매매)__실거래가_202010.csv" ,sep='|',encoding="cp949")
df = pd.read_excel("trimdata/apt200601.xlsx" )

In [9]:
df.columns

Index(['Unnamed: 0', '시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '계약년월', '계약일',
       '거래금액(만원)', '층', '건축년도', '도로명', '해제사유발생일'],
      dtype='object')

In [11]:
# 이전파일 건드리지 않으려면 반드시 copy로
df1 = df.copy()

# datetime으로 만들기

df1["년월"] = df1["계약년월"].copy()
#format=%Y%m 으로 포맷 바꿔주기.
df1["년월"] = pd.to_datetime(df1["년월"], format='%Y%m', errors='ignore')

df1["년월"]
# pd.to_datetime?
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25921 entries, 0 to 25920
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  25921 non-null  int64         
 1   시군구         25921 non-null  object        
 2   번지          25921 non-null  object        
 3   본번          25921 non-null  int64         
 4   부번          25921 non-null  int64         
 5   단지명         25921 non-null  object        
 6   전용면적(㎡)     25921 non-null  float64       
 7   계약년월        25921 non-null  int64         
 8   계약일         25921 non-null  int64         
 9   거래금액(만원)    25921 non-null  object        
 10  층           25921 non-null  int64         
 11  건축년도        25921 non-null  int64         
 12  도로명         25921 non-null  object        
 13  해제사유발생일     0 non-null      float64       
 14  년월          25921 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(7), object(5)
memory usage: 3.

In [18]:
df1["거래금액(만원)"]
## 왜안돼?
"""
df1["금액"] = pd.to_numeric(df1["거래금액(만원)"],errors ='coerce')
df1["금액"]
"""
df1["금액"] = pd.to_numeric(df1["거래금액(만원)"])

"""
에러가 나는 이유. 아마 콤마때문인거같음.
기본적으로 
 값 중에 몇 개가 실수로 숫자로 된 문자열이 아니라 문자로 된 문자열이 몇 개 포함되어 있다고 해봅시다. 이럴 경우 문자열을 숫자로 파싱할 수 없다면서 ValueError가 발생하는데요,

출처: https://rfriend.tistory.com/470 [R, Python 분석과 프로그래밍의 친구 (by R Friend)]
"""



ValueError: Unable to parse string "         4,200" at position 0

In [22]:
import re

In [30]:
# 문자열에서 제거하기

string = "Hey! What's up, bro?"
new_string = string.replace("!","").replace("?","").replace(",","")
#new_string = ''.join(filter(str.isalnum, string)) 
print(new_string)

Hey What's up bro


In [38]:

"""
# 특정 공백또는 문자 제거하기 
string = "Hey! What's up, bro?"
new_string = string.strip("W")
#df의 컬럼 a에 대해 적용 시킬 때
df['a'].str.strip()
#df 전체에 다 적용시킬 때
df.apply(lambda x: x.str.strip(), axis = 1)
print(new_string)
"""

Hey! What's up, bro?


In [42]:

# ,가 문자열에 있어서 to_numeric에서  인식이 안되므로 ,제거하가ㅣ 
    # 성공! replace던 strip이던 컬럼에 적용할땐 항상 .str. 을 써야함을 잊지말것. 
df1["금액"] =df1["거래금액(만원)"].copy()
df1["금액"] = df1["금액"].str.replace(",","")
df1["금액"]

0                 4200
1                 7500
2                 8500
3                12500
4                 5998
             ...      
25916             6400
25917             5500
25918             4500
25919             9000
25920             7200
Name: 금액, Length: 25921, dtype: object

In [44]:
df1["금액"] =pd.to_numeric(df1["금액"])
df1["금액"]

0         4200
1         7500
2         8500
3        12500
4         5998
         ...  
25916     6400
25917     5500
25918     4500
25919     9000
25920     7200
Name: 금액, Length: 25921, dtype: int64

In [12]:
#스트링 숫자로 바꿔주기.
#KeyError발생
# df_next["분양가격"] = pd.to_numeric(df_next["분양가격(㎡)"],errors ='coerce')
# 거래금액인데 잘못씀..
df1["금액"] = pd.to_numeric(df1["거래금액(만원)"],errors ='coerce')

df1["건축년도"]=pd.to_numeric(df1["건축년도"],errors ='coerce')

df1["면적"]=pd.to_numeric(df1["전용면적(㎡)"],errors ='coerce')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25921 entries, 0 to 25920
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  25921 non-null  int64         
 1   시군구         25921 non-null  object        
 2   번지          25921 non-null  object        
 3   본번          25921 non-null  int64         
 4   부번          25921 non-null  int64         
 5   단지명         25921 non-null  object        
 6   전용면적(㎡)     25921 non-null  float64       
 7   계약년월        25921 non-null  int64         
 8   계약일         25921 non-null  int64         
 9   거래금액(만원)    25921 non-null  object        
 10  층           25921 non-null  int64         
 11  건축년도        25921 non-null  int64         
 12  도로명         25921 non-null  object        
 13  해제사유발생일     0 non-null      float64       
 14  년월          25921 non-null  datetime64[ns]
 15  금액          6 non-null      float64       
 16  면적          25921 non-

In [14]:
df1["건축년도"]


0        1997
1        1999
2        1999
3        1999
4        1999
         ... 
25916    1994
25917    1994
25918    1994
25919    1994
25920    1994
Name: 건축년도, Length: 25921, dtype: int64

In [ ]:
df.index[0:14]

In [ ]:
#스트링 숫자로 바꿔주기.
#KeyError발생
# df_next["분양가격"] = pd.to_numeric(df_next["분양가격(㎡)"],errors ='coerce')
# 거래금액인데 잘못씀..
df1["금액"] = pd.to_numeric(df1["거래금액(만원)"],errors ='coerce')

df1["건축년도"]=pd.to_numeric(df1["건축년도"],errors ='coerce')

df1["면적"]=pd.to_numeric(df1["전용면적(㎡)"],errors ='coerce')
df1.info()

계획. 
1. 지역명으로 분리 replace사용.

2. 구별로 분리


In [ ]:
df['동찾기'] = df["법정동"].str.contains("창", case=False, na=False).astype(int)
df['동찾기']

In [ ]:
#해제사유등 필요없는것 제거하기.

# axis=1을 쓰면 열을 지우고 df.drop([0, 1], axis=0) 처럼 0을쓰면 인덱스(행)을 지움.
# 주의 행의 이름은 B랑 C가 아닌 0 1인데 이는 스트링이 아니므로 ''없이 하면됨.
df2 = df1.drop(["해제사유발생일", "계약년월","단지명","계약일","층","도로명","번지","본번","거래금액(만원)","전용면적(㎡)","부번"]
               ,axis=1).copy()


In [ ]:
df2['지역'] = "서울"
df2.info()

In [ ]:
 #지역으로 나누기.
df2["서울"] = df2["시군구"].str.contains("서울", case=False, na=False).astype(int)
df2["서울"].sum()
df2["대전"] = df2["시군구"].str.contains("대전", case=False, na=False).astype(int)
df2["대전"].sum()

location =[]
location = ["강원도",     
            "경기도"  ,  
            "경상남도" ,    
            "경상북도"  ,   
            "광주광역"     ,
            "대구광역"  ,
            "대전광역"   ,
            "부산광역"   ,
            "서울특별"    ,
            "세종특별",
             "울산광역"    ,
            "인천광역"    ,
            "전라남도" ,    
            "전라북도"  ,   
            "제주특별"    ,
            "충청남도" ,   
            "충청북도"  ] 

for i in location :
    df2[i] =  df2["시군구"].str.contains(i, case=False, na=False).astype(int)
    df2.loc[(df2[i] ==  1) , '지역'] = i

df2["지역"].value_counts()

In [ ]:
df2["강원도"].sum()+df2["경기도"].sum()+df2["경상남도"].sum()+df2["경상북도"].sum()+df2["광주광역"].sum()+df2["대구광역"].sum()+df2["대전광역"].sum()+df2["부산광역"].sum()+df2["서울특별"].sum()+df2["세종특별"].sum()+df2["울산광역"].sum()+df2["인천광역"].sum()+df2["전라남도"].sum()+df2["전라북도"].sum()+df2["제주특별"].sum()+df2["충청남도"].sum() +df2["충청북도"].sum()



#뭔가잘못됐네.. 뭔가 누락됨. 해결 제주특별 이었음..

In [47]:
df2["세종특별"].sum()

88

In [49]:
df2["강남구"]

0        0
1        0
2        0
3        0
4        0
        ..
25916    0
25917    0
25918    0
25919    0
25920    0
Name: 강남구, Length: 25921, dtype: int32

In [ ]:
###문제 
###문제 
###문제 
###문제 


df2["면적"]

# value error 
# if df2["면적"] <= 85 & df2["면적"] >= 60 :
#     df2["크기"] = "소형 "
#type error 
# if df2["면적"] <= 85
#  df2["크기"] = "소형 "
# if df2["면적"] <= 60
#     df2["크기"] =  1 

# print df2 if df2["면적"] <= 60
# #뭘 못하고있는가? 범주로 쪼개서 새로운 변수 만들기.
#  [x for x in raw_fe.columns if x not in Y_colname+X_remove]

df2["면적"]>= 60 #이건됨 

#이건 타입에러가남. 
# df2["면적"]>= 60  & df2["면적"] <= 85 

# value error 
# df2["면적"]>= 60.0 and   df2["면적"] <= 85.0
df2["크기"] = df2["면적"] <= 85.0 
# if  df2["면적"]>= 60.0 :
#     if  df2["면적"] <= 85.0 :
#         df2["크기"] = "중형"
#     else :
#         df2["크기"] = "대형"
# df2["확인"] = 1 if df2["크기"]==True #안됨



 
#boolean 인덱싱 불리언 인덱싱

# names = np.array[df2["면적"]]
df2["소형"] = df2["면적"]<= 60
df2["대형"] =  df2["면적"]> 85
# if df2["대형"]==False and df2["소형"] ==False :
#     df2["중형"]
    
#df2["면적"]> 60  and  df2["면적"] <85
type(df2["소형"])

df2.loc[(df2['면적'] >  60) & (df2['면적'] <= 85), '크기'] = '중형' 
df2.loc[(df2['면적'] <=  60),'크기'] = '소형'
df2.loc[(df2['면적'] > 85), '크기'] = '대형'

# https://datatofish.com/if-condition-in-pandas-dataframe/


# df2.count() if df2['크기'] =="대형" 
#질문 '' 와 " " 의 차이? 
#갯수 카운트하기
df2['크기'].value_counts()
df2['제주특별'].value_counts() 
# df2.info()

In [81]:
#강남 송파 서초 3구 따로 묶어보기 ..abs
df2["강남구"] = df2["시군구"].str.contains("서울특별시 강남구", case=False, na=False).astype(int)
df2["서초구"] = df2["시군구"].str.contains("서울특별시 서초구", case=False, na=False).astype(int)
df2["송파구"] = df2["시군구"].str.contains("서울특별시 송파구", case=False, na=False).astype(int)
df2.loc[(df2['강남구'] == 1) | (df2['서초구'] == 1) |  (df2['송파구'] == 1) , "강남3구"] = 1

#Nan 0으로 만들기
df2["강남3구"] = df2["강남3구"].fillna(0).copy()

#확인하기 
df2["강남3구"].value_counts()

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
25916    0.0
25917    0.0
25918    0.0
25919    0.0
25920    0.0
Name: 강남3구, Length: 25921, dtype: float64

In [4]:
df2["강남3구"].value_counts()


0.0    24158
1.0     1763
Name: 강남3구, dtype: int64

In [51]:
#강남3구를 제외한 서울 만들기 
df2.loc[(df2['서울특별'] == 1) & (df2['강남3구'] != 1) ,"서울강남제외"] = 1
df2["서울강남제외"] = df2["서울강남제외"].fillna(0).copy()
df2["서울강남제외"].value_counts()
df2["서울강남제외"].mean()


0.15790285868600748

In [48]:
#잘나왔나 확인해보기

df2["서울특별"].sum()

#만약 어떤 조건에 맞는 값들만을 찾고싶으면
noseoul = df2["지역"] != "서울특별"
noseoul.value_counts()


True     20065
False     5856
Name: 지역, dtype: int64

In [50]:
# 두개 조건은 안되나?  
"""에러 
noseoul = df2["지역"] == "서울특별"  & df2["강남3구"] != 1
noseoul.value_counts()

TypeError: Cannot perform 'rand_' with a dtyped [float64] array and scalar of type [bool]
"""

TypeError: Cannot perform 'rand_' with a dtyped [float64] array and scalar of type [bool]

In [52]:
z = 0
if  df2["강남3구"].loc() ==1 :
    z = df2["금액"].mean()
    z
z

0

In [26]:
# groupby로 통계값들을 얻기위한 작업 이미 근데 있네..?

# 왜 에러날까?
"""
for i in location:
    if df2[location[i]]==1 :
        df2["지역명"] = i
df2["지역명"]

뜬 에러 :  TypeError: list indices must be integers or slices, not str
일단 루프가 각 스트링에 대해 돌아가므로 당연히 그냥 i가들어가야됨. 
근데 일단 지역명할때 이프문 필요없는듯?

loc를 사용해봅시다.  
"""
"""
df2["지역명"] = []
for i in location:

z= df2[location[1]]==1
location[1]
"""
""
df2["지역"]

0         강원도
1         강원도
2         강원도
3         강원도
4         강원도
         ... 
25916    충청북도
25917    충청북도
25918    충청북도
25919    충청북도
25920    충청북도
Name: 지역, Length: 25921, dtype: object

In [55]:
## 해당 지역으로 평균내기
# ?하고명령어쓰면 설명뜸. 
# ?df2.groupby
df2.groupby(["지역"])["금액"].mean()
df2.groupby(["강남3구"])["금액"].mean()
# df2.groupby(["서울강남제외"])["금액"].mean()


강남3구
0.0    18421.007658
1.0    69244.115712
Name: 금액, dtype: float64

조건문 여러개 달기 .

if (cond1 AND/OR COND2) AND/OR (cond3 AND/OR cond4):
    code1
else:
    code2

데이터에서 조건문 여러개 달기

df1 = df[(df.a != -1) & (df.b != -1)]
또는
df["A"][(df["B"] > 50) & (df["C"] == 900)]
위에꺼는  에러가 날 수도있으므로 특정값을 찾는 .loc 사용을 추천.
df.loc[(df["B"] > 50) & (df["C"] == 900), "A"] *= 1000 

https://stackoverflow.com/questions/15315452/selecting-with-complex-criteria-from-pandas-dataframe

In [57]:
df["지역"]

KeyError: '지역'

In [65]:
#여러조건으로 dataframe수정하기.

df2["새로운"]=0
df2.loc[(df2["지역"] =="서울특별" ) & (df2["강남3구"] != 1), "새로운"] = 1

df2["새로운"].sum()

4093

to do list


시계열데이터 추출해내기
    뭘 추출해야되는가?
        금액의 평균 분산
        지역에따라서 가져와야할듯
        어떤식으로 할것인가? 평균 분산등 데이터 한다음 그냥 각각마다 column만들고 비시계열인 부문지우고 특정조건만족하는식으로하면될듯.

        즉 column들을 무수히 늘린형태로 만든다음  row를 한줄만남기고  가야될듯

    여러 시계열 데이터들을 한번에 frame에 추출하기
        데이터를 불러와서 각각의 dfi에 집어넣어놓고 작업을하는게맞을거같긴한데..
    시계열 데이터합치기 
        달마다의 시계열데이터들을 각각의 frame에 저장해두고 이걸 melt로 녹이는 방식을 써야도리듯?


In [ ]:

# 파일 불러오기 
df = pd.read_excel("trimdata/apt200601.xlsx" )

# 이전파일 건드리지 않으려면 반드시 copy로
df1 = df.copy()

# datetime으로 만들기

df1["년월"] = df1["계약년월"].copy()
#format=%Y%m 으로 포맷 바꿔주기.
df1["년월"] = pd.to_datetime(df1["년월"], format='%Y%m', errors='ignore')

df1["년월"]
# pd.to_datetime?
# df1.info()


# ,가 문자열에 있어서 to_numeric에서  인식이 안되므로 ,제거하기 
    # 성공! replace던 strip이던 컬럼에 적용할땐 항상 .str. 을 써야함을 잊지말것. 
df1["금액"] =df1["거래금액(만원)"].copy()
df1["금액"] = df1["금액"].str.replace(",","")


# 문자열인 값 숫자로 변환하기 
df1["금액"] =pd.to_numeric(df1["금액"])
df1["면적"]=pd.to_numeric(df1["전용면적(㎡)"],errors ='coerce')
df1["건축년도"]=pd.to_numeric(df1["건축년도"],errors ='coerce')


#해제사유등 필요없는것 제거하기.

# axis=1을 쓰면 열을 지우고 df.drop([0, 1], axis=0) 처럼 0을쓰면 인덱스(행)을 지움.
# 주의 행의 이름은 B랑 C가 아닌 0 1인데 이는 스트링이 아니므로 ''없이 하면됨.
df2 = df1.drop(["해제사유발생일", "계약년월","단지명","계약일","층","도로명","번지","본번","거래금액(만원)","전용면적(㎡)","부번"]
               ,axis=1).copy()
df2


    
#크기로 나누기
df2.loc[(df2['면적'] >  60) & (df2['면적'] <= 85), '크기'] = '중형' 
df2.loc[(df2['면적'] <=  60),'크기'] = '소형'
df2.loc[(df2['면적'] > 85), '크기'] = '대형'
#확인 
df2['크기'].value_counts()


 #지역으로 나누기.
df2["서울"] = df2["시군구"].str.contains("서울", case=False, na=False).astype(int)
df2["서울"].sum()
df2["대전"] = df2["시군구"].str.contains("대전", case=False, na=False).astype(int)
df2["대전"].sum()

location =[]
location = ["강원도",     
            "경기도"  ,  
            "경상남도" ,    
            "경상북도"  ,   
            "광주광역"     ,
            "대구광역"  ,
            "대전광역"   ,
            "부산광역"   ,
            "서울특별"    ,
            "세종특별",
             "울산광역"    ,
            "인천광역"    ,
            "전라남도" ,    
            "전라북도"  ,   
            "제주특별"    ,
            "충청남도" ,   
            "충청북도"  ] 

for i in location :
    df2[i] =  df2["시군구"].str.contains(i, case=False, na=False).astype(int)
    df2.loc[(df2[i] ==  1) , '지역'] = i


#확인
df2["지역"].value_counts()
df2["강원도"].sum()+df2["경기도"].sum()+df2["경상남도"].sum()+df2["경상북도"].sum()+df2["광주광역"].sum()+df2["대구광역"].sum()+df2["대전광역"].sum()+df2["부산광역"].sum()+df2["서울특별"].sum()+df2["세종특별"].sum()+df2["울산광역"].sum()+df2["인천광역"].sum()+df2["전라남도"].sum()+df2["전라북도"].sum()+df2["제주특별"].sum()+df2["충청남도"].sum() +df2["충청북도"].sum()


#강남 송파 서초 3구 따로 묶어보기 ..abs
df2["강남구"] = df2["시군구"].str.contains("서울특별시 강남구", case=False, na=False).astype(int)
df2["서초구"] = df2["시군구"].str.contains("서울특별시 서초구", case=False, na=False).astype(int)
df2["송파구"] = df2["시군구"].str.contains("서울특별시 송파구", case=False, na=False).astype(int)
df2.loc[(df2['강남구'] == 1) | (df2['서초구'] == 1) |  (df2['송파구'] == 1) , "강남3구"] = 1

#Nan 0으로 만들기
df2["강남3구"] = df2["강남3구"].fillna(0).copy()

#확인하기 
df2["강남3구"].value_counts()


# 전체관측치와 각도별 관측치 갯수의합이 같은지 확인하기.
df2["강원도"].sum()+df2["경기도"].sum()+df2["경상남도"].sum()+df2["경상북도"].sum()+df2["광주광역"].sum()+df2["대구광역"].sum()+df2["대전광역"].sum()+df2["부산광역"].sum()+df2["서울특별"].sum()+df2["세종특별"].sum()+df2["울산광역"].sum()+df2["인천광역"].sum()+df2["전라남도"].sum()+df2["전라북도"].sum()+df2["제주특별"].sum()+df2["충청남도"].sum() +df2["충청북도"].sum()

#결과 확인
df2.info()

# 결과 출력하기 

In [132]:
df2.groupby(["지역","크기"])["금액"].mean().unstack().round()


크기,대형,소형,중형
지역,,,
강원도,13083.0,4802.0,8661.0
경기도,48854.0,12439.0,22490.0
경상남도,21554.0,6928.0,11991.0
경상북도,13508.0,4504.0,8261.0
광주광역,17346.0,5749.0,10917.0
대구광역,25533.0,8683.0,15049.0
대전광역,30536.0,7549.0,14870.0
부산광역,21505.0,8118.0,12083.0
서울특별,80705.0,24957.0,40114.0


In [19]:

df3["평균금액"] = pd.df2["금액"].mean()
df3["년월"] = 200601
df3["년월"] = pd.to_datetime(df3["년월"], format='%Y%m', errors='ignore')
df3.info()
df3.head()

AttributeError: module 'pandas' has no attribute 'df2'

In [129]:
## 설명 시계열 데이터 변환작업.
column_names = []
df3 = pd.DataFrame(columns = column_names)
df3 = df2.groupby(["지역","크기"])["금액"].mean().unstack().round()
df3.columns = ["대형평균","소형평균","중형평균"]
df3 = df3.reindex(columns=["소형평균","중형평균","대형평균"])
df3


df4 = df2.groupby(["지역","크기"])["금액"].std().unstack().round()
df4.columns = ["대형표준편차","소형표준편차","중형표준편차"]
df4 = df4.reindex(columns=["소형표준편차","중형표준편차","대형표준편차"])
df4

column_names = []
df5 = pd.DataFrame(columns = column_names)
df5["지역평균"] = df2.groupby(["지역"])["금액"].mean().round().copy()

df6 = pd.DataFrame(columns = column_names)
df6["지역표준편차"] = df2.groupby(["지역"])["금액"].std().round().copy()

df_last = pd.concat([df5,df3,df6,df4] , axis=1) #axis=1을 해야 옆에다 붙임즉 column에 붙임. 디폴트는 axis=0 이고 이렇게하면 row에 붙이는것. 
df_last

,지역평균,소형평균,중형평균,대형평균,지역표준편차,소형표준편차,중형표준편차,대형표준편차
지역,,,,,,,,
강원도,6361.0,4802.0,8661.0,13083.0,3503.0,1958.0,3211.0,4855.0
경기도,23442.0,12439.0,22490.0,48854.0,19567.0,6869.0,12291.0,26989.0
경상남도,9554.0,6928.0,11991.0,21554.0,6218.0,2555.0,5571.0,11758.0
경상북도,6666.0,4504.0,8261.0,13508.0,3687.0,1924.0,3311.0,4405.0
광주광역,9047.0,5749.0,10917.0,17346.0,4835.0,1615.0,3135.0,5915.0
대구광역,12717.0,8683.0,15049.0,25533.0,6625.0,2422.0,4572.0,8605.0
대전광역,14715.0,7549.0,14870.0,30536.0,9741.0,2889.0,5300.0,10816.0
부산광역,11749.0,8118.0,12083.0,21505.0,6291.0,2711.0,4117.0,8956.0
서울특별,42495.0,24957.0,40114.0,80705.0,32687.0,16706.0,21030.0,43356.0


In [131]:
df_last[""]

KeyError: '지역'

In [88]:
column_names = []
df7 = pd.DataFrame(columns = column_names)
df7["평균금액"] = df2.["금액"].mean().round().copy()
df7


,평균금액
지역,
강원도,6361.0
경기도,23442.0
경상남도,9554.0
경상북도,6666.0
광주광역,9047.0
대구광역,12717.0
대전광역,14715.0
부산광역,11749.0
서울특별,42495.0


In [111]:
column_names = []
dfnat = pd.DataFrame(columns = column_names)
dfnat = df2.groupby(["크기"])["금액"].mean().round().copy()
dfnat = dfnat.T
dfnat


크기
대형    51808.0
소형    11752.0
중형    22382.0
Name: 금액, dtype: float64

In [137]:
# 리스트를 작성해서 밑에 열로 붙이기
# 중요: 차원을 맞춰야함.
nat = []

nat.append(df2["금액"].mean())
nat.append(df2.groupby(["크기"])["금액"].mean()["소형"])
nat.append(df2.groupby(["크기"])["금액"].mean()["중형"])
nat.append(df2.groupby(["크기"])["금액"].mean()["대형"])
nat.append(df2["금액"].std())
nat.append(df2.groupby(["크기"])["금액"].std()["소형"])
nat.append(df2.groupby(["크기"])["금액"].std()["중형"])
nat.append(df2.groupby(["크기"])["금액"].std()["대형"])
nat

df_last.loc[17] = nat



In [69]:
df6 = df2.groupby(["지역"])["금액"].mean().round().copy()
df6

지역
강원도      6361.0
경기도     23442.0
경상남도     9554.0
경상북도     6666.0
광주광역     9047.0
대구광역    12717.0
대전광역    14715.0
부산광역    11749.0
서울특별    42495.0
세종특별     6701.0
울산광역     8855.0
인천광역    11617.0
전라남도     5574.0
전라북도     6662.0
제주특별    10214.0
충청남도     8797.0
충청북도     7982.0
Name: 금액, dtype: float64

In [56]:
### 컬럼 순서바꾸기

order =df5.columns.tolist()
order

order[-1:]

['중형표준편차']

In [46]:
df3 = df2.copy()
p = pd.pivot_table(df3, index=["지역","크기"],values="금액")
#짤려서 하는거 해결하는 .loc 행을 기준으로 가져올때 지정함. 
#loc[] 를 빈칸으로 남기면 에러남.
p.loc["서울특별","소형"]



금액    24957.311347
Name: (서울특별, 소형), dtype: float64

In [48]:
?pd.pivot_table

Signature:
pd.pivot_table(
    data,
    values=None,
    index=None,
    columns=None,
    aggfunc='mean',
    fill_value=None,
    margins=False,
    dropna=True,
    margins_name='All',
    observed=False,
) -> 'DataFrame'
Docstring:
Create a spreadsheet-style pivot table as a DataFrame.

The levels in the pivot table will be stored in MultiIndex objects
(hierarchical indexes) on the index and columns of the result DataFrame.

Parameters
----------
data : DataFrame
values : column to aggregate, optional
index : column, Grouper, array, or list of the previous
    If an array is passed, it must be the same length as the data. The
    list can contain any of the other types (except list).
    Keys to group by on the pivot table index.  If an array is passed,
    it is being used as the same manner as column values.
columns : column, Grouper, array, or list of the previous
    If an array is passed, it must be the same length as the data. The
    list can contain any of the other types 

In [136]:
df_last.iloc[1]

지역평균      23442.0
소형평균      12439.0
중형평균      22490.0
대형평균      48854.0
지역표준편차    19567.0
소형표준편차     6869.0
중형표준편차    12291.0
대형표준편차    26989.0
Name: 경기도, dtype: float64

In [157]:
###  설명 시계열 데이터 변환작업 


df3 = df2.groupby(["지역","크기"])["금액"].mean().unstack().round()
df3.columns = ["대형평균","소형평균","중형평균"]
df3 = df3.reindex(columns=["소형평균","중형평균","대형평균"])
df3


df4 = df2.groupby(["지역","크기"])["금액"].std().unstack().round()
df4.columns = ["대형표준편차","소형표준편차","중형표준편차"]
df4 = df4.reindex(columns=["소형표준편차","중형표준편차","대형표준편차"])
df4

column_names = []
df5 = pd.DataFrame(columns = column_names)
df5["지역평균"] = df2.groupby(["지역"])["금액"].mean().round().copy()

df6 = pd.DataFrame(columns = column_names)
df6["지역표준편차"] = df2.groupby(["지역"])["금액"].std().round().copy()

df_last = pd.concat([df5,df3,df6,df4] , axis=1) #axis=1을 해야 옆에다 붙임즉 column에 붙임. 디폴트는 axis=0 이고 이렇게하면 row에 붙이는것. 


df_last
# 인덱스 초기화하기
df_r = df_last.reset_index()
df_r 




,지역,지역평균,소형평균,중형평균,대형평균,지역표준편차,소형표준편차,중형표준편차,대형표준편차
0,강원도,6361.0,4802.0,8661.0,13083.0,3503.0,1958.0,3211.0,4855.0
1,경기도,23442.0,12439.0,22490.0,48854.0,19567.0,6869.0,12291.0,26989.0
2,경상남도,9554.0,6928.0,11991.0,21554.0,6218.0,2555.0,5571.0,11758.0
3,경상북도,6666.0,4504.0,8261.0,13508.0,3687.0,1924.0,3311.0,4405.0
4,광주광역,9047.0,5749.0,10917.0,17346.0,4835.0,1615.0,3135.0,5915.0
5,대구광역,12717.0,8683.0,15049.0,25533.0,6625.0,2422.0,4572.0,8605.0
6,대전광역,14715.0,7549.0,14870.0,30536.0,9741.0,2889.0,5300.0,10816.0
7,부산광역,11749.0,8118.0,12083.0,21505.0,6291.0,2711.0,4117.0,8956.0
8,서울특별,42495.0,24957.0,40114.0,80705.0,32687.0,16706.0,21030.0,43356.0
9,세종특별,6701.0,5776.0,9734.0,12000.0,2615.0,1620.0,2961.0,NaN


In [159]:
#전국 붙이기 
nat = []
nat =["전국"]
nat.append(df2["금액"].mean())
nat.append(df2.groupby(["크기"])["금액"].mean()["소형"])
nat.append(df2.groupby(["크기"])["금액"].mean()["중형"])
nat.append(df2.groupby(["크기"])["금액"].mean()["대형"])
nat.append(df2["금액"].std())
nat.append(df2.groupby(["크기"])["금액"].std()["소형"])
nat.append(df2.groupby(["크기"])["금액"].std()["중형"])
nat.append(df2.groupby(["크기"])["금액"].std()["대형"])

nat

#시간축만들기. 여기 루프넣어야함. 
df_r.loc[17] = nat

df_r["년월"] = 200601
df_r["년월"] = pd.to_datetime(df_r["년월"], format='%Y%m', errors='ignore')


df_r

,지역,지역평균,소형평균,중형평균,대형평균,지역표준편차,소형표준편차,중형표준편차,대형표준편차,년월
0,강원도,6361.000000,4802.000000,8661.000000,13083.000000,3503.000000,1958.000000,3211.00000,4855.000000,2006-01-01
1,경기도,23442.000000,12439.000000,22490.000000,48854.000000,19567.000000,6869.000000,12291.00000,26989.000000,2006-01-01
2,경상남도,9554.000000,6928.000000,11991.000000,21554.000000,6218.000000,2555.000000,5571.00000,11758.000000,2006-01-01
3,경상북도,6666.000000,4504.000000,8261.000000,13508.000000,3687.000000,1924.000000,3311.00000,4405.000000,2006-01-01
4,광주광역,9047.000000,5749.000000,10917.000000,17346.000000,4835.000000,1615.000000,3135.00000,5915.000000,2006-01-01
5,대구광역,12717.000000,8683.000000,15049.000000,25533.000000,6625.000000,2422.000000,4572.00000,8605.000000,2006-01-01
6,대전광역,14715.000000,7549.000000,14870.000000,30536.000000,9741.000000,2889.000000,5300.00000,10816.000000,2006-01-01
7,부산광역,11749.000000,8118.000000,12083.000000,21505.000000,6291.000000,2711.000000,4117.00000,8956.000000,2006-01-01
8,서울특별,42495.000000,24957.000000,40114.000000,80705.000000,32687.000000,16706.000000,21030.00000,43356.000000,2006-01-01
9,세종특별,6701.000000,5776.000000,9734.000000,12000.000000,2615.000000,1620.000000,2961.00000,NaN,2006-01-01


In [165]:
## 세종시 결측값 왜 생기는지 이해해보기
## 관측치가 한개라 NAN임 이건 일괄적으로 1으로 고칩시다.
df2.groupby(["지역","크기"])["금액"].var().unstack().round()


size = df2['크기'] == '대형'
location = df2['지역'] =="세종특별"
subset_df = df2[size & location]
subset_df



,Unnamed: 0,시군구,건축년도,년월,금액,면적,크기,서울,대전,강원도,...,인천광역,전라남도,전라북도,제주특별,충청남도,충청북도,강남구,서초구,송파구,강남3구
21198,21214,세종특별자치시 조치원읍 죽림리,1996,2006-01-01,12000,114.91,대형,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0



# Code Summary



In [1]:
import requests, bs4
import pandas as pd
from lxml import html #html.lxml 로 써진 것. 즉 html.lxml 로 써야하는 명령어를 lxml만써도 작동하게해줌.
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from pprint import pprint
import numpy as np

In [ ]:
%ls APTdata

In [2]:

# 파일 불러오기 
df = pd.read_excel("trimdata/apt200601.xlsx" )

# 이전파일 건드리지 않으려면 반드시 copy로
df1 = df.copy()

# datetime으로 만들기

df1["년월"] = df1["계약년월"].copy()
#format=%Y%m 으로 포맷 바꿔주기.
df1["년월"] = pd.to_datetime(df1["년월"], format='%Y%m', errors='ignore')

df1["년월"]
# pd.to_datetime?
# df1.info()


# ,가 문자열에 있어서 to_numeric에서  인식이 안되므로 ,제거하기 
    # 성공! replace던 strip이던 컬럼에 적용할땐 항상 .str. 을 써야함을 잊지말것. 
df1["금액"] =df1["거래금액(만원)"].copy()
df1["금액"] = df1["금액"].str.replace(",","")


# 문자열인 값 숫자로 변환하기 
df1["금액"] =pd.to_numeric(df1["금액"])
df1["면적"]=pd.to_numeric(df1["전용면적(㎡)"],errors ='coerce')
df1["건축년도"]=pd.to_numeric(df1["건축년도"],errors ='coerce')


#해제사유등 필요없는것 제거하기.

# axis=1을 쓰면 열을 지우고 df.drop([0, 1], axis=0) 처럼 0을쓰면 인덱스(행)을 지움.
# 주의 행의 이름은 B랑 C가 아닌 0 1인데 이는 스트링이 아니므로 ''없이 하면됨.
df2 = df1.drop(["해제사유발생일", "계약년월","단지명","계약일","층","도로명","번지","본번","거래금액(만원)","전용면적(㎡)","부번"]
               ,axis=1).copy()
df2


    
#크기로 나누기
df2.loc[(df2['면적'] >  60) & (df2['면적'] <= 85), '크기'] = '중형' 
df2.loc[(df2['면적'] <=  60),'크기'] = '소형'
df2.loc[(df2['면적'] > 85), '크기'] = '대형'
#확인 
df2['크기'].value_counts()


 #지역으로 나누기.
df2["서울"] = df2["시군구"].str.contains("서울", case=False, na=False).astype(int)
df2["서울"].sum()
df2["대전"] = df2["시군구"].str.contains("대전", case=False, na=False).astype(int)
df2["대전"].sum()

location =[]
location = ["강원도",     
            "경기도"  ,  
            "경상남도" ,    
            "경상북도"  ,   
            "광주광역"     ,
            "대구광역"  ,
            "대전광역"   ,
            "부산광역"   ,
            "서울특별"    ,
            "세종특별",
             "울산광역"    ,
            "인천광역"    ,
            "전라남도" ,    
            "전라북도"  ,   
            "제주특별"    ,
            "충청남도" ,   
            "충청북도"  ] 

for i in location :
    df2[i] =  df2["시군구"].str.contains(i, case=False, na=False).astype(int)
    df2.loc[(df2[i] ==  1) , '지역'] = i


#확인
df2["지역"].value_counts()
df2["강원도"].sum()+df2["경기도"].sum()+df2["경상남도"].sum()+df2["경상북도"].sum()+df2["광주광역"].sum()+df2["대구광역"].sum()+df2["대전광역"].sum()+df2["부산광역"].sum()+df2["서울특별"].sum()+df2["세종특별"].sum()+df2["울산광역"].sum()+df2["인천광역"].sum()+df2["전라남도"].sum()+df2["전라북도"].sum()+df2["제주특별"].sum()+df2["충청남도"].sum() +df2["충청북도"].sum()


#강남 송파 서초 3구 따로 묶어보기 ..abs
df2["강남구"] = df2["시군구"].str.contains("서울특별시 강남구", case=False, na=False).astype(int)
df2["서초구"] = df2["시군구"].str.contains("서울특별시 서초구", case=False, na=False).astype(int)
df2["송파구"] = df2["시군구"].str.contains("서울특별시 송파구", case=False, na=False).astype(int)
df2.loc[(df2['강남구'] == 1) | (df2['서초구'] == 1) |  (df2['송파구'] == 1) , "강남3구"] = 1

#Nan 0으로 만들기
df2["강남3구"] = df2["강남3구"].fillna(0).copy()

#확인하기 
df2["강남3구"].value_counts()


# 전체관측치와 각도별 관측치 갯수의합이 같은지 확인하기.
df2["강원도"].sum()+df2["경기도"].sum()+df2["경상남도"].sum()+df2["경상북도"].sum()+df2["광주광역"].sum()+df2["대구광역"].sum()+df2["대전광역"].sum()+df2["부산광역"].sum()+df2["서울특별"].sum()+df2["세종특별"].sum()+df2["울산광역"].sum()+df2["인천광역"].sum()+df2["전라남도"].sum()+df2["전라북도"].sum()+df2["제주특별"].sum()+df2["충청남도"].sum() +df2["충청북도"].sum()

#결과 확인
df2.info()

# 결과 출력하기 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25921 entries, 0 to 25920
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  25921 non-null  int64         
 1   시군구         25921 non-null  object        
 2   건축년도        25921 non-null  int64         
 3   년월          25921 non-null  datetime64[ns]
 4   금액          25921 non-null  int64         
 5   면적          25921 non-null  float64       
 6   크기          25921 non-null  object        
 7   서울          25921 non-null  int32         
 8   대전          25921 non-null  int32         
 9   강원도         25921 non-null  int32         
 10  지역          25921 non-null  object        
 11  경기도         25921 non-null  int32         
 12  경상남도        25921 non-null  int32         
 13  경상북도        25921 non-null  int32         
 14  광주광역        25921 non-null  int32         
 15  대구광역        25921 non-null  int32         
 16  대전광역        25921 non-

In [91]:
# 전체관측치와 각도별 관측치 갯수의합이 같은지 확인하기.
df2["강원도"].sum()+df2["경기도"].sum()+df2["경상남도"].sum()+df2["경상북도"].sum()+df2["광주광역"].sum()+df2["대구광역"].sum()+df2["대전광역"].sum()+df2["부산광역"].sum()+df2["서울특별"].sum()+df2["세종특별"].sum()+df2["울산광역"].sum()+df2["인천광역"].sum()+df2["전라남도"].sum()+df2["전라북도"].sum()+df2["제주특별"].sum()+df2["충청남도"].sum() +df2["충청북도"].sum()


25921

In [86]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25921 entries, 0 to 25920
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  25921 non-null  int64         
 1   시군구         25921 non-null  object        
 2   건축년도        25921 non-null  int64         
 3   년월          25921 non-null  datetime64[ns]
 4   금액          25921 non-null  int64         
 5   면적          25921 non-null  float64       
 6   서울          25921 non-null  int32         
 7   대전          25921 non-null  int32         
 8   강원도         25921 non-null  int32         
 9   지역          25921 non-null  object        
 10  경기도         25921 non-null  int32         
 11  경상남도        25921 non-null  int32         
 12  경상북도        25921 non-null  int32         
 13  광주광역        25921 non-null  int32         
 14  대구광역        25921 non-null  int32         
 15  대전광역        25921 non-null  int32         
 16  부산광역        25921 non-

In [167]:
###  설명 시계열 데이터 변환작업 


df3 = df2.groupby(["지역","크기"])["금액"].mean().unstack().round()
df3.columns = ["대형평균","소형평균","중형평균"]
df3 = df3.reindex(columns=["소형평균","중형평균","대형평균"])
df3


df4 = df2.groupby(["지역","크기"])["금액"].std().unstack().round()
df4.columns = ["대형표준편차","소형표준편차","중형표준편차"]
df4 = df4.reindex(columns=["소형표준편차","중형표준편차","대형표준편차"])
df4

column_names = []
df5 = pd.DataFrame(columns = column_names)
df5["지역평균"] = df2.groupby(["지역"])["금액"].mean().round().copy()

df6 = pd.DataFrame(columns = column_names)
df6["지역표준편차"] = df2.groupby(["지역"])["금액"].std().round().copy()

df_last = pd.concat([df5,df3,df6,df4] , axis=1) #axis=1을 해야 옆에다 붙임즉 column에 붙임. 디폴트는 axis=0 이고 이렇게하면 row에 붙이는것. 


df_last
# 인덱스 초기화하기
df_r = df_last.reset_index()
df_r 


,지역,지역평균,소형평균,중형평균,대형평균,지역표준편차,소형표준편차,중형표준편차,대형표준편차
0,강원도,6361.0,4802.0,8661.0,13083.0,3503.0,1958.0,3211.0,4855.0
1,경기도,23442.0,12439.0,22490.0,48854.0,19567.0,6869.0,12291.0,26989.0
2,경상남도,9554.0,6928.0,11991.0,21554.0,6218.0,2555.0,5571.0,11758.0
3,경상북도,6666.0,4504.0,8261.0,13508.0,3687.0,1924.0,3311.0,4405.0
4,광주광역,9047.0,5749.0,10917.0,17346.0,4835.0,1615.0,3135.0,5915.0
5,대구광역,12717.0,8683.0,15049.0,25533.0,6625.0,2422.0,4572.0,8605.0
6,대전광역,14715.0,7549.0,14870.0,30536.0,9741.0,2889.0,5300.0,10816.0
7,부산광역,11749.0,8118.0,12083.0,21505.0,6291.0,2711.0,4117.0,8956.0
8,서울특별,42495.0,24957.0,40114.0,80705.0,32687.0,16706.0,21030.0,43356.0
9,세종특별,6701.0,5776.0,9734.0,12000.0,2615.0,1620.0,2961.0,NaN


In [168]:
#전국 붙이기 
nat = []
nat =["전국"]
nat.append(df2["금액"].mean())
nat.append(df2.groupby(["크기"])["금액"].mean()["소형"])
nat.append(df2.groupby(["크기"])["금액"].mean()["중형"])
nat.append(df2.groupby(["크기"])["금액"].mean()["대형"])
nat.append(df2["금액"].std())
nat.append(df2.groupby(["크기"])["금액"].std()["소형"])
nat.append(df2.groupby(["크기"])["금액"].std()["중형"])
nat.append(df2.groupby(["크기"])["금액"].std()["대형"])

nat

#시간축만들기. 여기 루프넣어야함. 
df_r.loc[17] = nat

df_r["년월"] = 200601
df_r["년월"] = pd.to_datetime(df_r["년월"], format='%Y%m', errors='ignore')


df_r

,지역,지역평균,소형평균,중형평균,대형평균,지역표준편차,소형표준편차,중형표준편차,대형표준편차,년월
0,강원도,6361.000000,4802.000000,8661.000000,13083.000000,3503.000000,1958.000000,3211.00000,4855.000000,2006-01-01
1,경기도,23442.000000,12439.000000,22490.000000,48854.000000,19567.000000,6869.000000,12291.00000,26989.000000,2006-01-01
2,경상남도,9554.000000,6928.000000,11991.000000,21554.000000,6218.000000,2555.000000,5571.00000,11758.000000,2006-01-01
3,경상북도,6666.000000,4504.000000,8261.000000,13508.000000,3687.000000,1924.000000,3311.00000,4405.000000,2006-01-01
4,광주광역,9047.000000,5749.000000,10917.000000,17346.000000,4835.000000,1615.000000,3135.00000,5915.000000,2006-01-01
5,대구광역,12717.000000,8683.000000,15049.000000,25533.000000,6625.000000,2422.000000,4572.00000,8605.000000,2006-01-01
6,대전광역,14715.000000,7549.000000,14870.000000,30536.000000,9741.000000,2889.000000,5300.00000,10816.000000,2006-01-01
7,부산광역,11749.000000,8118.000000,12083.000000,21505.000000,6291.000000,2711.000000,4117.00000,8956.000000,2006-01-01
8,서울특별,42495.000000,24957.000000,40114.000000,80705.000000,32687.000000,16706.000000,21030.00000,43356.000000,2006-01-01
9,세종특별,6701.000000,5776.000000,9734.000000,12000.000000,2615.000000,1620.000000,2961.00000,NaN,2006-01-01


In [169]:
## 세종시 결측값 왜 생기는지 이해해보기
## 관측치가 한개라 NAN임 이건 일괄적으로 1으로 고칩시다.
df2.groupby(["지역","크기"])["금액"].var().unstack().round()


size = df2['크기'] == '대형'
location = df2['지역'] =="세종특별"
subset_df = df2[size & location]
subset_df


,Unnamed: 0,시군구,건축년도,년월,금액,면적,크기,서울,대전,강원도,...,인천광역,전라남도,전라북도,제주특별,충청남도,충청북도,강남구,서초구,송파구,강남3구
21198,21214,세종특별자치시 조치원읍 죽림리,1996,2006-01-01,12000,114.91,대형,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
